In [1]:
from transformers import * 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import random 
import re 
import math 
from tqdm import tqdm 
import sklearn 
import torch 
import torch.nn as nn 
import torch.functional as f
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from sklearn.model_selection import train_test_split 
import time 
import datetime


In [2]:
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv") 

train.shape, test.shape, train['label'].nunique()


((174304, 13), (43576, 12), 46)

In [3]:
zero_train = train[train['label'] == 0] 
nonzero_train = train[train['label'] != 0]

In [4]:
train_df = nonzero_train

In [5]:
def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean


def split_text(s, overlap = 20, chunk_size = 50): 
    total = [] 
    partial = [] 
    if len(s.split()) // (chunk_size - overlap) > 0:  
        n = len(s.split()) // (chunk_size - overlap) 
    else: 
        n = 1 
    for w in range(n): 
        if w == 0: 
            partial = s.split()[:chunk_size] 
            total.append(" ".join(partial)) 
        else:  
            partial = s.split()[w*(chunk_size - overlap):w*(chunk_size - overlap) + chunk_size]
            total.append(" ".join(partial)) 
    return total


In [6]:
train_df['요약문_내용'] = train_df['요약문_연구목표'] + train_df['요약문_연구내용'] + train_df['요약문_기대효과'] 
test['요약문_내용'] = test['요약문_연구목표'] + test['요약문_연구내용'] + test['요약문_기대효과']


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
train_df['사업명'].fillna('NAN',inplace=True) 
train_df['사업_부처명'].fillna('NAN',inplace=True) 
train_df['내역사업명'].fillna('NAN',inplace=True) 
train_df['과제명'].fillna('NAN',inplace=True) 
train_df['요약문_한글키워드'].fillna('NAN',inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [8]:
train_df['요약문_내용'].fillna('NAN',inplace=True) 
test['요약문_내용'].fillna('NAN',inplace=True)


In [9]:
contents = train_df['요약문_내용'].values 
feature1 = train_df['사업명'].values 
feature2 = train_df['사업_부처명'].values 
feature3 = train_df['내역사업명'].values 
feature4 = train_df['과제명'].values 
feature5 = train_df['요약문_한글키워드'].values 
feature6 = train_df['label'].values 

train_data = {'사업명':[],'사업_부처명':[],'내역사업명':[],'과제명':[],'한글키워드':[],'요약문':[],'label':[]} 

for i in tqdm(range(contents.shape[0]), position = 0, leave = True): 
    sample = str(contents[i]) 
    splitted_text = split_text(clean_text(sample)) 
    for t in splitted_text: 
        train_data['요약문'].append(t) 
        train_data['사업명'].append(clean_text(str(feature1[i])))
        train_data['사업_부처명'].append(clean_text(str(feature2[i]))) 
        train_data['내역사업명'].append(clean_text(str(feature3[i]))) 
        train_data['과제명'].append(clean_text(str(feature4[i])))  
        train_data['한글키워드'].append(feature5[i]) # no cleaning for this one
        train_data['label'].append(feature6[i])
        

100%|██████████| 31733/31733 [00:19<00:00, 1635.75it/s]


In [10]:
train_data = pd.DataFrame(train_data)
train_data.head(5)

,사업명,사업_부처명,내역사업명,과제명,한글키워드,요약문,label
0,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",새로운 해충분류군의 동정기술 개발 및 유입확산 추적 가 외래 및 돌발해충의 발생조사...,24
1,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",의 돌발 및 외래해충 다 외래 및 돌발해충의 유전적 다양성 조사 시험곤충 나 의 해...,24
2,핵융합기초연구,과학기술정보통신부,개인기초,국제 핵융합 재료조사시설 의 안정적 운전을 위한 고강도 빔 물리 연구,"국제 핵융합 재료 조사 시설,공간전하 효과,고강도 빔,PIC 시뮬레이션,빔 후광입자",기존 가속기 설계 및 운전에 있어서 엄밀하게 취급 되지 않았던 공간전하 효과 선형 ...,2
3,핵융합기초연구,과학기술정보통신부,개인기초,국제 핵융합 재료조사시설 의 안정적 운전을 위한 고강도 빔 물리 연구,"국제 핵융합 재료 조사 시설,공간전하 효과,고강도 빔,PIC 시뮬레이션,빔 후광입자",재료조사시설 개발에 적용한다 고강도 빔에서 플라즈마 물리 효과가 고려된 코드를 도입...,2
4,핵융합기초연구,과학기술정보통신부,개인기초,국제 핵융합 재료조사시설 의 안정적 운전을 위한 고강도 빔 물리 연구,"국제 핵융합 재료 조사 시설,공간전하 효과,고강도 빔,PIC 시뮬레이션,빔 후광입자",측정 가능하고 매우 높은 일반적으로 이상 를 갖는 빔 진단 시스템을 설계하고 테스트...,2


In [16]:
## Now we tokenize each data and make sure they all lie within the 512 tokenization range 
## if not check how many have token length greater than 512 

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator") 

def electra_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences, not "really" necessary for now    
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:512] 
        attention_mask = attention_mask[:512] 
        token_type_id = token_type_id[:512]  
        print("Long Text!! Truncating to the first 512 tokens!")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        token_type_id = token_type_id + [0]*(512 - len(token_type_id))  
        
    return input_id, attention_mask, token_type_id


 63%|██████▎   | 185543/296378 [06:30<03:14, 570.41it/s]

In [12]:
# define important hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 30
VALID_SPLIT = 0.1 
MAX_LEN = 512 # max token size for BERT, ELECTRA

In [13]:
train_data['data'] = train_data['사업명'] + " " + train_data['사업_부처명'] + " " + train_data['내역사업명'] + " " + train_data['과제명'] + " " + train_data['한글키워드'] + " " + train_data['요약문'] 

train_data.head(2)


,사업명,사업_부처명,내역사업명,과제명,한글키워드,요약문,label,data
0,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",새로운 해충분류군의 동정기술 개발 및 유입확산 추적 가 외래 및 돌발해충의 발생조사...,24,농업기초기반연구 농촌진흥청 농산물안전성연구 유전정보를 활용한 새로운 해충 분류군 동...
1,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",의 돌발 및 외래해충 다 외래 및 돌발해충의 유전적 다양성 조사 시험곤충 나 의 해...,24,농업기초기반연구 농촌진흥청 농산물안전성연구 유전정보를 활용한 새로운 해충 분류군 동...


In [14]:
train_text = train_data['data'].values 
train_labels = train_data['label'].values 
train_labels -= 1 # the labels must start from zero 

In [17]:
N = train_data.shape[0] 

input_ids = np.zeros((N, MAX_LEN),dtype=int)
attention_masks = np.zeros((N, MAX_LEN),dtype=int)
token_type_ids = np.zeros((N, MAX_LEN),dtype=int) 
labels = np.zeros((N),dtype=int)

for i in tqdm(range(N), position=0, leave=True): 
    try:
        cur_str = train_text[i]
        cur_label = train_labels[i]
        input_id, attention_mask, token_type_id = electra_tokenizer(cur_str, MAX_LEN=MAX_LEN) 
        input_ids[i,] = input_id 
        attention_masks[i,] = attention_mask 
        token_type_ids[i,] = token_type_id
        labels[i] = cur_label 
    except Exception as e: 
        print(e)
        print(cur_str)
        pass   

 34%|███▍      | 100637/296378 [03:20<06:17, 518.79it/s]

Long Text!! Truncating to the first 512 tokens!


 38%|███▊      | 113025/296378 [03:44<05:48, 525.56it/s]

Long Text!! Truncating to the first 512 tokens!
Long Text!! Truncating to the first 512 tokens!
Long Text!! Truncating to the first 512 tokens!


 54%|█████▍    | 160079/296378 [05:16<04:04, 557.80it/s]

...

 73%|███████▎  | 215723/296378 [07:07<02:30, 534.30it/s]

Long Text!! Truncating to the first 512 tokens!


 77%|███████▋  | 226802/296378 [07:29<02:01, 573.79it/s]

Long Text!! Truncating to the first 512 tokens!


100%|██████████| 296378/296378 [09:50<00:00, 502.07it/s]


In [18]:
input_ids = torch.tensor(input_ids, dtype=int)
attention_masks = torch.tensor(attention_masks, dtype=int)
token_type_ids = torch.tensor(token_type_ids, dtype=int) 
labels = torch.tensor(labels, dtype=int)

In [19]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, random_state = 42, test_size = VALID_SPLIT, stratify=labels) 

train_attention_mask, val_attention_mask, _, _ = train_test_split(attention_masks, labels, random_state = 42, test_size = VALID_SPLIT, stratify=labels) 

train_token_ids, val_token_ids, _, _ = train_test_split(token_type_ids, labels, random_state = 42, test_size = VALID_SPLIT, stratify=labels) 


train_inputs.shape, train_attention_mask.shape, train_token_ids.shape, train_labels.shape, val_inputs.shape, val_attention_mask.shape, val_token_ids.shape, val_labels.shape


(torch.Size([266740, 512]),
 torch.Size([266740, 512]),
 torch.Size([266740, 512]),
 torch.Size([266740]),
 torch.Size([29638, 512]),
 torch.Size([29638, 512]),
 torch.Size([29638, 512]),
 torch.Size([29638]))

In [20]:
train_data = TensorDataset(train_inputs, train_attention_mask, train_token_ids, train_labels) 
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE) 

validation_data = TensorDataset(val_inputs, val_attention_mask, val_token_ids, val_labels) 
validation_sampler = SequentialSampler(validation_data) 
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

In [21]:
# if label is nonzero, check which label the text belongs to 
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=45)
model.cuda()


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [22]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

epochs = 10 

total_steps = len(train_dataloader) * epochs 

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

def flat_accuracy(preds, labels): 
    pred_flat = np.argmax(preds, axis=1).flatten() 
    labels_flat = labels.flatten() 
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


device = torch.device("cuda")

best_val_loss = 1e9 


# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 20 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print('  current average loss = {}'.format(total_loss / step))

        # 배치를 GPU에 넣음
                
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=b_token_type_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        
        # 로스 구함
        loss = outputs[0]
        
        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=b_token_type_ids, 
                            attention_mask=b_input_mask, 
                            labels = b_labels)
    
        
        loss = outputs[0] 
        logits = outputs[1] 
        
        # 로스 구함 
        eval_loss += loss.item() 
        
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    
    avg_val_loss = eval_loss / len(validation_dataloader)             
    print("  Average validation loss: {}".format(avg_val_loss))
    print("  Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0))) 
    
    
    if best_val_loss > avg_val_loss: 
        best_val_loss = avg_val_loss 
        torch.save(model.state_dict(), "ELECTRA_NONZERO_MODEL_" + str(epoch_i + 1))  

print("")
print("Training complete!")



======== Epoch 1 / 10 ========
Training...
  Batch    20  of  8,336.    Elapsed: 0:00:20.
  current average loss = 3.7181488513946532
  Batch    40  of  8,336.    Elapsed: 0:00:38.
  current average loss = 3.640039545297623
  Batch    60  of  8,336.    Elapsed: 0:00:57.
  current average loss = 3.5740784128506977
  Batch    80  of  8,336.    Elapsed: 0:01:16.
  current average loss = 3.5237556427717207
  Batch   100  of  8,336.    Elapsed: 0:01:35.
  current average loss = 3.4765479135513306
  Batch   120  of  8,336.    Elapsed: 0:01:54.
  current average loss = 3.4018272240956624
  Batch   140  of  8,336.    Elapsed: 0:02:13.
  current average loss = 3.35550776379449
  Batch   160  of  8,336.    Elapsed: 0:02:32.
  current average loss = 3.3049743682146073
  Batch   180  of  8,336.    Elapsed: 0:02:51.
  current average loss = 3.2541176941659717
  Batch   200  of  8,336.    Elapsed: 0:03:10.
  current average loss = 3.1960127186775207
  Batch   220  of  8,336.    Elapsed: 0:03:29.
  

  Batch 1,820  of  8,336.    Elapsed: 0:28:47.
  current average loss = 1.4899020508422955
  Batch 1,840  of  8,336.    Elapsed: 0:29:07.
  current average loss = 1.4810584403736435
  Batch 1,860  of  8,336.    Elapsed: 0:29:26.
  current average loss = 1.4732759190022304
  Batch 1,880  of  8,336.    Elapsed: 0:29:45.
  current average loss = 1.4651525852765808
  Batch 1,900  of  8,336.    Elapsed: 0:30:04.
  current average loss = 1.4576366298528094
  Batch 1,920  of  8,336.    Elapsed: 0:30:23.
  current average loss = 1.4493822227154547
  Batch 1,940  of  8,336.    Elapsed: 0:30:42.
  current average loss = 1.4424961586511627
  Batch 1,960  of  8,336.    Elapsed: 0:31:01.
  current average loss = 1.4348311925649035
  Batch 1,980  of  8,336.    Elapsed: 0:31:20.
  current average loss = 1.4280483207994639
  Batch 2,000  of  8,336.    Elapsed: 0:31:39.
  current average loss = 1.4207371064797043
  Batch 2,020  of  8,336.    Elapsed: 0:31:58.
  current average loss = 1.412812853486526


  Batch 3,640  of  8,336.    Elapsed: 0:57:39.
  current average loss = 1.0302634445377268
  Batch 3,660  of  8,336.    Elapsed: 0:57:58.
  current average loss = 1.0270272315545974
  Batch 3,680  of  8,336.    Elapsed: 0:58:17.
  current average loss = 1.0241276914436046
  Batch 3,700  of  8,336.    Elapsed: 0:58:36.
  current average loss = 1.0216353083563012
  Batch 3,720  of  8,336.    Elapsed: 0:58:55.
  current average loss = 1.01834846295176
  Batch 3,740  of  8,336.    Elapsed: 0:59:14.
  current average loss = 1.0151084003601483
  Batch 3,760  of  8,336.    Elapsed: 0:59:33.
  current average loss = 1.012064331544048
  Batch 3,780  of  8,336.    Elapsed: 0:59:52.
  current average loss = 1.0088468357723541
  Batch 3,800  of  8,336.    Elapsed: 1:00:11.
  current average loss = 1.006024190183533
  Batch 3,820  of  8,336.    Elapsed: 1:00:30.
  current average loss = 1.0030774810473333
  Batch 3,840  of  8,336.    Elapsed: 1:00:49.
  current average loss = 1.0001476206836135
  B

  Batch 5,460  of  8,336.    Elapsed: 1:26:34.
  current average loss = 0.8131377228668758
  Batch 5,480  of  8,336.    Elapsed: 1:26:53.
  current average loss = 0.8113732290012341
  Batch 5,500  of  8,336.    Elapsed: 1:27:12.
  current average loss = 0.8095912645513361
  Batch 5,520  of  8,336.    Elapsed: 1:27:31.
  current average loss = 0.807718743559351
  Batch 5,540  of  8,336.    Elapsed: 1:27:50.
  current average loss = 0.8058139518010917
  Batch 5,560  of  8,336.    Elapsed: 1:28:09.
  current average loss = 0.8039654297553164
  Batch 5,580  of  8,336.    Elapsed: 1:28:28.
  current average loss = 0.8020930126229281
  Batch 5,600  of  8,336.    Elapsed: 1:28:47.
  current average loss = 0.8003892989443349
  Batch 5,620  of  8,336.    Elapsed: 1:29:06.
  current average loss = 0.7989524496447468
  Batch 5,640  of  8,336.    Elapsed: 1:29:25.
  current average loss = 0.7973511277506432
  Batch 5,660  of  8,336.    Elapsed: 1:29:44.
  current average loss = 0.7953903990577209


  Batch 7,280  of  8,336.    Elapsed: 1:55:27.
  current average loss = 0.6803114802354006
  Batch 7,300  of  8,336.    Elapsed: 1:55:46.
  current average loss = 0.6790962947999472
  Batch 7,320  of  8,336.    Elapsed: 1:56:05.
  current average loss = 0.6780608077826171
  Batch 7,340  of  8,336.    Elapsed: 1:56:24.
  current average loss = 0.6769071369830326
  Batch 7,360  of  8,336.    Elapsed: 1:56:43.
  current average loss = 0.6755518765940391
  Batch 7,380  of  8,336.    Elapsed: 1:57:01.
  current average loss = 0.6745299689375813
  Batch 7,400  of  8,336.    Elapsed: 1:57:20.
  current average loss = 0.6734070765800976
  Batch 7,420  of  8,336.    Elapsed: 1:57:39.
  current average loss = 0.6720660939787557
  Batch 7,440  of  8,336.    Elapsed: 1:57:58.
  current average loss = 0.6708466410000998
  Batch 7,460  of  8,336.    Elapsed: 1:58:17.
  current average loss = 0.6695648335462221
  Batch 7,480  of  8,336.    Elapsed: 1:58:36.
  current average loss = 0.6683318857372524

...

  Batch 8,300  of  8,336.    Elapsed: 2:11:05.
  current average loss = 0.010390068475062434
  Batch 8,320  of  8,336.    Elapsed: 2:11:24.
  current average loss = 0.010396533392871386

  Average training loss: 0.010389390428048022
  Training epoch took: 2:11:38

Running Validation...
  Average validation loss: 0.015465405289177876
  Accuracy: 0.9931566882416397
  Validation took: 0:04:53

======== Epoch 10 / 10 ========
Training...
  Batch    20  of  8,336.    Elapsed: 0:00:19.
  current average loss = 0.008627438172334223
  Batch    40  of  8,336.    Elapsed: 0:00:38.
  current average loss = 0.008196690420663798
  Batch    60  of  8,336.    Elapsed: 0:00:57.
  current average loss = 0.009874823879484514
  Batch    80  of  8,336.    Elapsed: 0:01:16.
  current average loss = 0.009579739454329684
  Batch   100  of  8,336.    Elapsed: 0:01:35.
  current average loss = 0.010020737882587127
  Batch   120  of  8,336.    Elapsed: 0:01:54.
  current average loss = 0.009534085397899617
  Ba

  Batch 1,700  of  8,336.    Elapsed: 0:26:51.
  current average loss = 0.00925085829135188
  Batch 1,720  of  8,336.    Elapsed: 0:27:09.
  current average loss = 0.00919350856010867
  Batch 1,740  of  8,336.    Elapsed: 0:27:28.
  current average loss = 0.009150393195792778
  Batch 1,760  of  8,336.    Elapsed: 0:27:47.
  current average loss = 0.009132235805980765
  Batch 1,780  of  8,336.    Elapsed: 0:28:06.
  current average loss = 0.00913237227342823
  Batch 1,800  of  8,336.    Elapsed: 0:28:25.
  current average loss = 0.009130958156608054
  Batch 1,820  of  8,336.    Elapsed: 0:28:44.
  current average loss = 0.00915714322520496
  Batch 1,840  of  8,336.    Elapsed: 0:29:03.
  current average loss = 0.009133949495840763
  Batch 1,860  of  8,336.    Elapsed: 0:29:22.
  current average loss = 0.00910312917714828
  Batch 1,880  of  8,336.    Elapsed: 0:29:41.
  current average loss = 0.00906659524295158
  Batch 1,900  of  8,336.    Elapsed: 0:30:00.
  current average loss = 0.00

  Batch 3,480  of  8,336.    Elapsed: 0:54:57.
  current average loss = 0.009013946979134692
  Batch 3,500  of  8,336.    Elapsed: 0:55:16.
  current average loss = 0.008986613662074627
  Batch 3,520  of  8,336.    Elapsed: 0:55:36.
  current average loss = 0.008944431310487744
  Batch 3,540  of  8,336.    Elapsed: 0:55:55.
  current average loss = 0.008921083085807486
  Batch 3,560  of  8,336.    Elapsed: 0:56:14.
  current average loss = 0.008878964780874769
  Batch 3,580  of  8,336.    Elapsed: 0:56:33.
  current average loss = 0.008864865356002425
  Batch 3,600  of  8,336.    Elapsed: 0:56:52.
  current average loss = 0.008854665871673812
  Batch 3,620  of  8,336.    Elapsed: 0:57:11.
  current average loss = 0.008898186739624385
  Batch 3,640  of  8,336.    Elapsed: 0:57:30.
  current average loss = 0.008904249213578673
  Batch 3,660  of  8,336.    Elapsed: 0:57:49.
  current average loss = 0.008876246501702932
  Batch 3,680  of  8,336.    Elapsed: 0:58:08.
  current average loss 

  Batch 5,260  of  8,336.    Elapsed: 1:23:06.
  current average loss = 0.008961361474655361
  Batch 5,280  of  8,336.    Elapsed: 1:23:24.
  current average loss = 0.008997961814154196
  Batch 5,300  of  8,336.    Elapsed: 1:23:43.
  current average loss = 0.009015747640641046
  Batch 5,320  of  8,336.    Elapsed: 1:24:02.
  current average loss = 0.009020612597379298
  Batch 5,340  of  8,336.    Elapsed: 1:24:21.
  current average loss = 0.0090235938863137
  Batch 5,360  of  8,336.    Elapsed: 1:24:40.
  current average loss = 0.009022256979142185
  Batch 5,380  of  8,336.    Elapsed: 1:24:59.
  current average loss = 0.009005679421021734
  Batch 5,400  of  8,336.    Elapsed: 1:25:18.
  current average loss = 0.008987826982485314
  Batch 5,420  of  8,336.    Elapsed: 1:25:37.
  current average loss = 0.008993706491502117
  Batch 5,440  of  8,336.    Elapsed: 1:25:56.
  current average loss = 0.008997206734633277
  Batch 5,460  of  8,336.    Elapsed: 1:26:15.
  current average loss = 

  Batch 7,140  of  8,336.    Elapsed: 1:52:48.
  current average loss = 0.008841637187949756
  Batch 7,160  of  8,336.    Elapsed: 1:53:07.
  current average loss = 0.008838853678050413
  Batch 7,180  of  8,336.    Elapsed: 1:53:26.
  current average loss = 0.008839596318335164
  Batch 7,200  of  8,336.    Elapsed: 1:53:45.
  current average loss = 0.008824809950094731
  Batch 7,220  of  8,336.    Elapsed: 1:54:04.
  current average loss = 0.00881333237696096
  Batch 7,240  of  8,336.    Elapsed: 1:54:23.
  current average loss = 0.008828221817841483
  Batch 7,260  of  8,336.    Elapsed: 1:54:42.
  current average loss = 0.008827648855787615
  Batch 7,280  of  8,336.    Elapsed: 1:55:01.
  current average loss = 0.008824570794608117
  Batch 7,300  of  8,336.    Elapsed: 1:55:20.
  current average loss = 0.008838717083815513
  Batch 7,320  of  8,336.    Elapsed: 1:55:39.
  current average loss = 0.008832528239138965
  Batch 7,340  of  8,336.    Elapsed: 1:55:58.
  current average loss =

In [23]:
best_val_loss

0.015465405289177876

Make Prediction using two different classifiers 

Algorithm: 
- Classify using zero/non-zero classifier 
- if nonzero, then classify which class it belongs (1-45) 

In [24]:
model0_ckpt = torch.load("ELECTRA_ZERO_MODEL_5") 
model1_ckpt = torch.load("ELECTRA_NONZERO_MODEL_9")

In [25]:
model0 = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels = 2)
model0.load_state_dict(model0_ckpt) 
model0.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [26]:
model1 = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels = 45)
model1.load_state_dict(model1_ckpt) 
model1.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [27]:
import torch.nn.functional as nnf

test['사업명'].fillna('NAN',inplace=True) 
test['사업_부처명'].fillna('NAN',inplace=True) 
test['내역사업명'].fillna('NAN',inplace=True) 
test['과제명'].fillna('NAN',inplace=True) 
test['요약문_한글키워드'].fillna('NAN',inplace=True)

In [33]:
test_contents = test['요약문_내용'].values 
test_feature1 = test['사업명'].values 
test_feature2 = test['사업_부처명'].values 
test_feature3 = test['내역사업명'].values 
test_feature4 = test['과제명'].values 
test_feature5 = test['요약문_한글키워드'].values 

predicted_classes = []

model0.eval()  
model1.eval() # for predicting non-zero classes 

for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):  
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i]))
    f5 = str(test_feature5[i]) 
    splitted = split_text(str(test_contents[i])) 
    probabilities = [] 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + text 
        input_id, attention_mask, token_type_id = electra_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        # convert to torch tensor 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        token_type_id = torch.tensor(token_type_id) 
        
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1,MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1,MAX_LEN)) 
        token_type_id = torch.reshape(token_type_id, (-1,MAX_LEN)) 
        
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        token_type_id = token_type_id.to(device) 
        
        with torch.no_grad(): 
            outputs = model0(input_id, 
                                 token_type_ids=token_type_id, 
                                 attention_mask=attention_mask) 
        
        logits = outputs[0] 
        prob = nnf.softmax(logits, dim=1).flatten()
        probabilities.append(prob)


    prob_sum = np.zeros(2) 
    for i in range(len(probabilities)): 
        for j in range(2): 
            prob_sum[j] += probabilities[i][j] 
            
    prob_sum /= len(probabilities) 
    
    predicted_class = np.argmax(prob_sum) 
    
    if predicted_class == 1: 
        # classify which one of 1-45 
        print("non-zero class prediction") 
        probabilities = [] 
        for text in splitted: 
            test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + text 
            input_id, attention_mask, token_type_id = electra_tokenizer(test_text, MAX_LEN=MAX_LEN) 
            # convert to torch tensor 
            input_id = torch.tensor(input_id)
            attention_mask = torch.tensor(attention_mask) 
            token_type_id = torch.tensor(token_type_id) 
        
            # reshape into (batch, MAX_LEN)
            input_id = torch.reshape(input_id, (-1,MAX_LEN)) 
            attention_mask = torch.reshape(attention_mask, (-1,MAX_LEN)) 
            token_type_id = torch.reshape(token_type_id, (-1,MAX_LEN)) 
        
            # move tensor to cuda 
            input_id = input_id.to(device) 
            attention_mask = attention_mask.to(device) 
            token_type_id = token_type_id.to(device) 
        
            with torch.no_grad(): 
                outputs = model1(input_id, 
                                 token_type_ids=token_type_id, 
                                 attention_mask=attention_mask) 
        
            logits = outputs[0] 
            prob = nnf.softmax(logits, dim=1).flatten()
            probabilities.append(prob)
            
        prob_sum = np.zeros(45) 
        for i in range(len(probabilities)):  
            for j in range(45): 
                prob_sum[j] += probabilities[i][j] 
        prob_sum /= len(probabilities) 
        predicted_class = np.argmax(prob_sum) 
        predicted_classes.append(predicted_class + 1) # zero based index 
        
    
    elif predicted_class == 0: 
        predicted_classes.append(predicted_class)

        


  0%|          | 10/43576 [00:04<4:24:58,  2.74it/s]

non-zero class prediction


  0%|          | 13/43576 [00:04<3:18:20,  3.66it/s]

non-zero class prediction


  0%|          | 20/43576 [00:06<2:35:11,  4.68it/s]

...

100%|█████████▉| 43569/43576 [3:20:06<00:01,  3.67it/s]

non-zero class prediction


100%|█████████▉| 43573/43576 [3:20:07<00:00,  4.55it/s]

non-zero class prediction


100%|██████████| 43576/43576 [3:20:07<00:00,  3.63it/s]


In [35]:

submission = pd.read_csv("sample_submission.csv")


submission.iloc[:,1] = predicted_classes 

submission.to_csv("zero_non_zero_ELECTRA.csv",index=False) 

submission

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
5,174309,0
6,174310,0
7,174311,0
8,174312,0
9,174313,14
